In [1]:
import re
import json
from urllib.parse import urlparse, parse_qs

import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
registry = pd.read_csv("20250627-0800-gleif-goldencopy-lei2-golden-copy.csv", low_memory=False)

In [3]:
relationships = pd.read_csv("20250627-0800-gleif-goldencopy-rr-golden-copy.csv", low_memory=False)

In [4]:
data = []
for _, row in relationships[relationships["Relationship.EndNode.NodeID"] == "UWJKFUJFZ02DKWI3RY53"].iterrows():
    df = registry[registry["LEI"] == row["Relationship.StartNode.NodeID"]]
    assert len(df) == 1
    data.append([
        row["Relationship.RelationshipType"],
        row["Relationship.Period.1.startDate"],
        row["Relationship.Period.1.endDate"],
        df.iloc[0]["Entity.LegalName"],
        df.iloc[0]["Entity.LegalAddress.FirstAddressLine"],
        df.iloc[0]["Entity.LegalAddress.City"],
        df.iloc[0]["Entity.LegalAddress.Region"] if isinstance(df.iloc[0]["Entity.LegalAddress.Region"], str) else df.iloc[0]["Entity.LegalAddress.Country"],
        df.iloc[0]["Entity.LegalAddress.PostalCode"],
        df.iloc[0]["Entity.HeadquartersAddress.FirstAddressLine"],
        df.iloc[0]["Entity.HeadquartersAddress.City"],
        df.iloc[0]["Entity.HeadquartersAddress.Region"] if isinstance(df.iloc[0]["Entity.HeadquartersAddress.Region"], str) else df.iloc[0]["Entity.HeadquartersAddress.Country"],
        df.iloc[0]["Entity.HeadquartersAddress.PostalCode"],
        df.iloc[0]["Entity.EntityStatus"],
        df.iloc[0]["Entity.EntityCreationDate"],
        row["Registration.ValidationReference"],
    ])

pd.DataFrame(data=data, columns=[
    "relationship", "start", "end", "subsidiary", "legal", "city", "region", "zip", "headquarters", "city", "region", "zip", "status", "creation", "validation"
]).sort_values(by="start")

,relationship,start,end,subsidiary,legal,city,region,zip,headquarters,city,region,zip,status,creation,validation
7,IS_DIRECTLY_CONSOLIDATED_BY,1992-02-19T18:30:00.001Z,NaN,COCA COLA INDIA PRIVATE LIMITED,"PLOT NO'S 1109-1110,",PUNE,IN-MH,412108,"1601, 16TH FLOOR",GURUGRAM,IN-HR,122002,ACTIVE,1992-02-19T18:30:00.001Z,NaN
8,IS_ULTIMATELY_CONSOLIDATED_BY,1992-02-19T18:30:00.001Z,NaN,COCA COLA INDIA PRIVATE LIMITED,"PLOT NO'S 1109-1110,",PUNE,IN-MH,412108,"1601, 16TH FLOOR",GURUGRAM,IN-HR,122002,ACTIVE,1992-02-19T18:30:00.001Z,NaN
6,IS_ULTIMATELY_CONSOLIDATED_BY,1997-02-13T18:30:00.001Z,NaN,HINDUSTAN COCA-COLA BEVERAGES PRIVATE LIMITED,"UNIT NOS.303 AND 304, 3RD FLOOR,",GURUGRAM,IN-HR,122011,"BRIGADE MAGNUM, 7TH,8TH &9TH FLOOR, (B-WING)",BENGALURU,IN-KA,560092,ACTIVE,1997-02-13T18:30:00.001Z,NaN
4,IS_DIRECTLY_CONSOLIDATED_BY,1997-02-20T18:30:00.001Z,NaN,HINDUSTAN COCA-COLA HOLDINGS PRIVATE LIMITED,"B-91,MAYAPURI INDUSTRIAL AREA",NEW DELHI,IN-DL,110064,"B-91,MAYAPURI INDUSTRIAL AREA",NEW DELHI,IN-DL,110064,ACTIVE,1997-02-20T18:30:00.001Z,https://investors.coca-colacompany.com/
5,IS_ULTIMATELY_CONSOLIDATED_BY,1997-02-20T18:30:00.001Z,NaN,HINDUSTAN COCA-COLA HOLDINGS PRIVATE LIMITED,"B-91,MAYAPURI INDUSTRIAL AREA",NEW DELHI,IN-DL,110064,"B-91,MAYAPURI INDUSTRIAL AREA",NEW DELHI,IN-DL,110064,ACTIVE,1997-02-20T18:30:00.001Z,https://investors.coca-colacompany.com/
14,IS_ULTIMATELY_CONSOLIDATED_BY,2018-01-01T00:00:00Z,2018-12-31T00:00:00Z,FRESH TRADING LIMITED,C/O FRUIT TOWERS,LONDON,GB-EAL,W10 5BU,C/O FRUIT TOWERS,LONDON,GB-EAL,W10 5BU,ACTIVE,NaN,https://beta.companieshouse.gov.uk/company/037...
11,IS_ULTIMATELY_CONSOLIDATED_BY,2021-01-01T00:00:00Z,2021-12-31T00:00:00Z,INNOCENT LIMITED,C/O FRUIT TOWERS,LONDON,GB-LND,W10 5BU,C/O FRUIT TOWERS,LONDON,GB-LND,W10 5BU,ACTIVE,2000-06-01T05:00:00Z,NaN
9,IS_DIRECTLY_CONSOLIDATED_BY,2022-01-01T00:00:00+01:00,2022-12-31T00:00:00+01:00,RED LIFE REINSURANCE LIMITED,Canon's Court,Hamilton,BM,HM 12,Canon's Court,Hamilton,BM,HM 12,ACTIVE,2014-01-13T13:00:00+01:00,NaN
10,IS_ULTIMATELY_CONSOLIDATED_BY,2022-01-01T00:00:00+01:00,2022-12-31T00:00:00+01:00,RED LIFE REINSURANCE LIMITED,Canon's Court,Hamilton,BM,HM 12,Canon's Court,Hamilton,BM,HM 12,ACTIVE,2014-01-13T13:00:00+01:00,NaN
18,IS_ULTIMATELY_CONSOLIDATED_BY,2023-01-01T00:00:00+00:00,2023-12-31T00:00:00+00:00,COCA-COLA BEVERAGES AFRICA,BUILDING 7 POMMERN STREET,GQEBERHA,ZA-EC,6001,BUILDING 7 POMMERN STREET,GQEBERHA,ZA-EC,6001,ACTIVE,2016-02-05T00:00:00+00:00,https://www.sec.gov/Archives/edgar/data/21344/...


In [5]:
len(registry) == len(registry.drop_duplicates("LEI"))

True

In [6]:
# for url in tofetch:
#     if url in webpage_to_cik or url.endswith(".pdf"):
#         continue
#     print(int(100 * len(webpage_to_cik) / len(tofetch)), url)
#     raw = requests.get(
#         url,
#         headers={
#             "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
#             "accept-language": "en-US,en;q=0.9",
#             "cache-control": "no-cache",
#             "pragma": "no-cache",
#             "priority": "u=0, i",
#             "sec-ch-ua": '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
#             "sec-ch-ua-mobile": "?0",
#             "sec-ch-ua-platform": '"Linux"',
#             "sec-fetch-dest": "document",
#             "sec-fetch-mode": "navigate",
#             "sec-fetch-site": "none",
#             "sec-fetch-user": "?1",
#             "upgrade-insecure-requests": "1",
#             "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36"
#         },
#     ).text
#     time.sleep(0.2)
#     tags = bs4.BeautifulSoup(raw)

#     different = tags.find("acronym", {"title": "Central Index Key"})
#     if different is not None:
#         webpage_to_cik[url] = re.match("([0-9]{10})", different.find_next_sibling().text).group(1)
#         continue

#     no_records = tags.find("h1")
#     if no_records is not None and no_records.text == "EDGAR Company Search Results":
#         continue

#     no_records = tags.find("a", {"class": "nav__link"})
#     if no_records is not None and no_records.text == "EDGAR":
#         continue

#     no_records = tags.find("center")
#     if no_records is not None and no_records.text == "No records matched your query.":
#         continue

#     no_records = tags.find("h2")
#     if no_records is not None and no_records.text == "There was an problem with your query.":
#         continue

#     webpage_to_cik[url] = tags.find("a", {"class": "search"}).text

webpage_to_cik = {
    "https://www.sec.gov/cgi-bin/series?company=proshares+trust": "0001174610",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Global+Allocation+Fund": "0000933691",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Advisor+Floating+Rate+High+Income+Fund&type=N-PX&sc=companyseries": "0000722574",
    "https://www.sec.gov/cgi-bin/series?company=PROFUNDS&sc=companyseries&ticker=&CIK=&type=N-PX": "0001039803",
    "https://www.sec.gov/cgi-bin/series?company=MFS+South+Carolina+Municipal+Bond+Fund&type=N-PX&sc=companyseries": "0000751656",
    "https://www.sec.gov/cgi-bin/series?company=Variable+Insurance+Products+Fund+V&sc=companyseries&ticker=&CIK=&type=N-PX": "0000823535",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+INVESTMENT+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000354046",
    "https://www.sec.gov/cgi-bin/series?company=AIM+TAX-EXEMPT+FUNDS+%28INVESCO+TAX-EXEMPT+FUNDS%29": "0000909466",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Advisor+Equity+Value+Fund": "0000722574",
    "https://www.sec.gov/cgi-bin/series?company=MFS+VARIABLE+INSURANCE+TRUST+II": "0000719269",
    "https://www.sec.gov/cgi-bin/series?company=RUSSELL+INVESTMENT+CO&sc=companyseries&ticker=&CIK=&type=N-PX": "0000351601",
    "https://www.sec.gov/cgi-bin/series?company=HARTFORD+INTERNATIONAL+OPPORTUNITIES+HLS+FUND": "0000861503",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+SELECT+PORTFOLIOS": "0000320351",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Lifetime+2020+Fund": "0001330967",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+SELECT+PORTFOLIOS&sc=companyseries&ticker=&CIK=&type=N-PX": "0000320351",
    "https://www.sec.gov/cgi-bin/series?sc=companyseries&type=N-PX&company=%09UBS+FUNDS+": "0000315774",
    "https://www.sec.gov/cgi-bin/series?company=ADVANCED+SERIES+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000814679",
    "https://www.sec.gov/cgi-bin/series?company=RUSSELL+INVESTMENT+CO": "0000351601",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+SUMMER+STREET+TRUST+": "0000225322",
    "https://www.sec.gov/cgi-bin/series?sc=companyseries&type=N-PX&company=LORD+ABBETT+DIVERSIFICATION+SHARES": "0001139819",
    "https://www.sec.gov/cgi-bin/series?company=VARIABLE+INSURANCE+PRODUCTS+FUND+III": "0000914036",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+SCHOOL+STREET+TRUST": "0000215829",
    "https://www.sec.gov/cgi-bin/series?company=%09ADVANCED+SERIES+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000814679",
    "https://www.sec.gov/cgi-bin/series?CIK=&sc=companyseries&type=N-PX&ticker=S000009323&Find=Search": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=PROFUNDS+": "0001039803",
    "https://www.sec.gov/cgi-bin/series?company=%09Variable+Insurance+Products+Fund+V": "0000823535",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+PURITAN+TRUST": "0000081205",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+International+Equity+Insights+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Value+Series&type=N-PX&sc=companyseries": "0000814679",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+SECURITIES+FUND&sc=companyseries&ticker=&CIK=&type=N-PX": "0000795422",
    "https://www.sec.gov/cgi-bin/series?company=VARIABLE+INSURANCE+PRODUCTS+FUND+IV": "0000720318",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+I&type=N-PX&sc=companyseries": "0000933691",
    "https://www.sec.gov/cgi-bin/series?company=HARTFORD+DISCIPLINED+EQUITY+HLS+FUND": "0001053425",
    "https://www.sec.gov/cgi-bin/series?company=Adviser+Managed+Trust": "0001502608",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Select+Portfolios": "0000320351",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+CONTRAFUND": "0000024238",
    "https://www.sec.gov/cgi-bin/series?company=MFS+MUNICIPAL+SERIES+TRUST": "0001392417",
    "https://www.sec.gov/cgi-bin/series?company=VARIABLE+INSURANCE+PRODUCTS+FUND+II": "0000914036",
    "https://www.sec.gov/cgi-bin/series?company=Russell+Investment+Funds&sc=companyseries&ticker=&CIK=&type=N-PX": "0000824036",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+IX+": "0000063075",
    "https://www.sec.gov/cgi-bin/series?company=DoubleLine+Commodity+Strategy+ETF": "0001886172",
    "https://www.sec.gov/cgi-bin/series?company=DoubleLine+Fortune+500+Equal+Weight+ETF": "0001886172",
    "https://www.sec.gov/cgi-bin/series?company=Exchange+Listed+Funds+Trust": "0001547950",
    "https://www.sec.gov/cgi-bin/series?company=J.P.+Morgan+Exchange-Traded+Fund+Trust&type=N-PX&sc=companyseries": "0001485894",
    "https://www.sec.gov/cgi-bin/series?company=Segall+Bryant+and+Hamill+Trust": "0001318342",
    "https://www.sec.gov/cgi-bin/series?company=Dimensional+ETF+Trust&type=N-PX&sc=companyseries": "0001816125",
    "https://www.sec.gov/cgi-bin/series?company=OAK+ASSOCIATES+FUNDS": "0001050918",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+ETF+Trust&type=N-PX&sc=companyseries": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Spinnaker+ETF+SerIES": "0001484018",
    "https://www.sec.gov/cgi-bin/series?company=DFA+INVESTMENT+TRUST+CO": "0000896162",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+ABERDEEN+STREET+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000880195",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Sustainable+Target+Date+2025+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0000880195",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Salem+Street+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=AIM+TREASURER%27S+SERIES+TRUST+%28INVESCO+TREASURER%27S+SERIES+TRUST%29&sc=companyseries&ticker=&CIK=&type=N-PX": "0000828806",
    "https://www.sec.gov/cgi-bin/series?company=AMERICAN+PENSION+INVESTORS+TRUST": "0000764859",
    "https://www.sec.gov/cgi-bin/series?company=Ecofin+Global+Sustainable+Listed+Infrastructure+Fund&type=N-PX&sc=companyseries": "0001511699",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Salem+Street+Trust&type=N-PX&sc=companyseries": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Series+Sustainable+Emerging+Markets+Fund": "0000744822",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Series+Sustainable+Non-U.S.+Developed+Markets+Fund": "0000744822",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Series+Sustainable+U.S.+Market+Fund": "0000225322",
    "https://www.sec.gov/cgi-bin/series?company=Kurv+Bank+of+America+%28BAC%29+1.5x+Long+Daily+ETF": "0001848758",
    "https://www.sec.gov/cgi-bin/series?company=Pacer+US+Small+Cap+Cash+Cows+Growth+Leaders+ETF": "0001616668",
    "https://www.sec.gov/cgi-bin/series?company=Principal+LifeTime+Hybrid+Income+Fund": "0000898745",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+BDC+Income+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000042702&type=N-PX": "0000798244",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000063026&type=N-PX": "0001710607",
    "https://www.sec.gov/cgi-bin/series?company=Kelly+Strategic+ETF+Trust": "0001873280",
    "https://www.sec.gov/cgi-bin/series?company=Syntax+Stratified+U.S.+Total+Market+Hedged+ETF": "0001580843",
    "https://www.sec.gov/cgi-bin/series?company=Tuttle+Capital+2X+All+Innovation+ETF": "0001771146",
    "https://www.sec.gov/cgi-bin/series?company=KraneShares+Rockefeller+Ocean+Engagement+ETF": "0001547576",
    "https://www.sec.gov/cgi-bin/series?company=ETF+Series+Solutions": "0001540305",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+ETF+Trust": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000001063&type=N-PX": "0000877670",
    "https://www.sec.gov/cgi-bin/series?company=Madison+ETFs+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001976877",
    "https://www.sec.gov/cgi-bin/series?company=GOLDMAN+SACHS+ETF+TRUST": "0001479026",
    "https://www.sec.gov/cgi-bin/series?company=tidal+trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001742912",
    "https://www.sec.gov/cgi-bin/series?company=Two+Roads+Shared+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001552947",
    "https://www.sec.gov/cgi-bin/series?company=KraneShares+Dynamic+Emerging+Markets+Strategy+ETF": "0001547576",
    "https://www.sec.gov/cgi-bin/series?company=KraneShares+Global+EM+Revenue+Leaders+Index+ETF": "0001547576",
    "https://www.sec.gov/cgi-bin/series?company=KraneShares+Global+Luxury+Index+ETF": "0001547576",
    "https://www.sec.gov/cgi-bin/series?company=ETF+Series+Solutions&type=N-PX&sc=companyseries": "0001540305",
    "https://www.sec.gov/cgi-bin/series?company=Municipal+Allocation+Portfolio": "0000835663",
    "https://www.sec.gov/cgi-bin/series?company=Dimensional+ETF+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001816125",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000080681&type=N-PX": "0001261788",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000080682&type=N-PX": "0001261788",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Mt.+Vernon+Street+Trust+&sc=companyseries&ticker=&CIK=&type=N-PX": "0000707823",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+Global+Real+Estate+Fund&type=N-PX&sc=companyseries": "0001112996",
    "https://www.sec.gov/cgi-bin/series?CIK=&sc=companyseries&type=N-PX&ticker=S000046562&Find=Search": "0000898745",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+Exchange-Traded+Self-Indexed+Fund+Trust": "0001657201",
    "https://www.sec.gov/cgi-bin/series?company=MANNING&type=N-PX&sc=companyseries": "0000751173",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+SmartRetirement*+Blend+2020+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Crypto+Industry+and+Digital+Payments+ETF": "0000945908",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Emerging+Markets+High+Yield+Bond+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000076095&type=N-PX": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=HOMESTEAD+FUNDS+TRUST": "0001768447",
    "https://www.sec.gov/cgi-bin/series?company=UNIFIED+SERIES+TRUST": "0001199046",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=S000054127&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=GAMCO+Global+Series+Funds%2C+Inc": "0000909504",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+NASDAQ+Composite+Index+Fund": "0000205323",
    "https://www.sec.gov/cgi-bin/series?company=WAYCROSS+FOCUSED+CORE+EQUITY+FUND": "0001815558",
    "https://www.sec.gov/cgi-bin/series?company=Listed+Funds+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001547950",
    "https://www.sec.gov/cgi-bin/series?company=Global+X+Founder-Run+Companies+ETF": "0001432353",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+EXCHANGE-TRADED+FUND+TRUST": "0001595386",
    "https://www.sec.gov/cgi-bin/series?company=Palm+Valley+Capital+Fund": "0001650149",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+MSCI+Materials+Index+ETF": "0000945908",
    "https://www.sec.gov/cgi-bin/series?CIK=&sc=companyseries&type=N-PX&ticker=S000068565&Find=Search": "0001547576",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+X": "0000783740",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+SALEM+STREET+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=AIM+SECTOR+FUNDS+%28INVESCO+SECTOR+FUNDS%29": "0000725781",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Emerging+Markets+Debt+Local+Currency+Fund&type=N-PX&sc=companyseries": "0000783740",
    "https://www.sec.gov/cgi-bin/series?company=KraneShares+Trust&type=N-PX&sc=companyseries": "0001452937",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000076098&type=N-PX": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=North+Square&sc=companyseries&ticker=&CIK=&type=N-PX": "0001408970",
    "https://www.sec.gov/cgi-bin/series?company=%09DoubleLine+ETF+Trust": "0001886172",
    "https://www.sec.gov/cgi-bin/series?company=PRINCIPAL+FUNDS%2C+INC": "0000898745",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Intermediate+Bond+Trust": "0001303608",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Digital+India+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+ETF+Trust": "0001479026",
    "https://www.sec.gov/cgi-bin/series?company=ALPS+Series+Trust": "0001558107",
    "https://www.sec.gov/cgi-bin/series?company=PRINCIPAL+VARIABLE+CONTRACTS+FUNDS+INC&sc=companyseries&ticker=&CIK=&type=N-PX": "0000012601",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Natural+Resources+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Social+Sentiment+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=AIM+INVESTMENT+FUNDS+%28INVESCO+INVESTMENT+FUNDS%29": "0000826644",
    "https://www.sec.gov/cgi-bin/series?company=DIREXION+SHARES+ETF+TRUST&type=N-PX&sc=companyseries": "0001424958",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+COVINGTON+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000945908",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+International+High+Yield+Bond+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Mortgage+REIT+Income+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=EQ+ADVISORS+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0001027263",
    "https://www.sec.gov/cgi-bin/series?company=Ultimus+Managers+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001545440",
    "https://www.sec.gov/cgi-bin/series?company=PUTNAM+FUNDS+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0001126087",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Access+Growth+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+SALEM+STREET+TRUST": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Salem+Street+Trust": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Moody%27s+Analytics+BBB+Corporate+Bond+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+MT+VERNON+STREET+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000707823",
    "https://www.sec.gov/cgi-bin/series?company=AQR+FUNDS": "0001444822",
    "https://www.sec.gov/cgi-bin/series?company=AIM+INTERNATIONAL+MUTUAL+FUNDS+%28INVESCO+INTERNATIONAL+MUTUAL+FUNDS%29": "0000880859",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+China+Bond+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=RBC+Funds+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001272950",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Sustainable+Low+Duration+Bond+Fund": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=The+Short+Term+Government+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0000926243",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+EXCHANGE-TRADED+FUND+TRUST+": "0001595386",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000076099&type=N-PX": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=Kelly+Strategic+ETF+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001873280",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Gold+Miners+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=PACIFIC+SELECT+FUND": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Multi-Manager+Non-Core+Fixed+Income+Fund": "0001557156",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+EXCHANGE-TRADED+FUND+TRUST&type=N-PX&sc=companyseries": "0001595386",
    "https://www.sec.gov/cgi-bin/series?company=Miller+Convertible+Bond+Fund": "0001414039",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+HEREFORD+STREET+TRUST": "0000917286",
    "https://www.sec.gov/cgi-bin/series?company=AIM+VARIABLE+INSURANCE+FUNDS": "0000896435",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Ultra-Short+Municipal+Fund&type=N-PX&sc=companyseries": "0001485894",
    "https://www.sec.gov/cgi-bin/series?company=SPDR+Index+Shares+Funds&sc=companyseries&ticker=&CIK=&type=N-PX": "0001168164",
    "https://www.sec.gov/cgi-bin/series?company=ProShares+Supply+Chain+Logistics+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001174610",
    "https://www.sec.gov/cgi-bin/series?company=AIM+SECTOR+FUNDS+%28INVESCO+SECTOR+FUNDS%29&type=N-PX&sc=companyseries": "0000725781",
    "https://www.sec.gov/cgi-bin/series?company=PROFUND+VP+INDUSTRIALS": "0001039803",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+IV": "0000063068",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Sustainable+Multi-Asset+Fund": "0000354046",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+SUMMER+STREET+TRUST": "0000225322",
    "https://www.sec.gov/cgi-bin/series?company=iNNOVATOR+etfS+TRUST": "0001415726",
    "https://www.sec.gov/cgi-bin/series?company=Eaton+Vance+NextShares+Trust+II": "0001614522",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Series+Opportunistic+Insights+Fund": "0000024238",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+Exchange-Traded+Fund+Trust": "0001595386",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+SmartRetirement+Blend+2055+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=VANGUARD+WHITEHALL+FUNDS+": "0001004655",
    "https://www.sec.gov/cgi-bin/series?company=PRINCIPAL+FUNDS%2C+INC.": "0000898745",
    "https://www.sec.gov/cgi-bin/series?company=Multi-Manager+International+Equity+Fund": "0000773757",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+SmartRetirement+Blend+2025+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=GOLDMAN+SACHS+ENERGY+INFRASTRUCTURE+FUND&sc=companyseries&ticker=&CIK=&type=N-PX": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=Goldman%20Sachs%20Trust&CIK=&start=1&count=500": "0000814679",
    "https://www.sec.gov/cgi-bin/series?sc=companyseries&type=N-PX&company=Two+Roads+Shared+Trust": "0001552947",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+High+Dividend+ETF": "0000945908",
    "https://www.sec.gov/cgi-bin/series?company=VALUE+LINE+FUNDS+VARIABLE+TRUST": "0000819978",
    "https://www.sec.gov/cgi-bin/series?company=LOCORR+INVESTMENT+TRUST&type=N-PX&sc=companyseries": "0001506768",
    "https://www.sec.gov/cgi-bin/series?company=AIM+COUNSELOR+SERIES+TRUST": "0001112996",
    "https://www.sec.gov/cgi-bin/series?company=ProShares+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001174610",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=S000059371&CIK=&type=N-PX": "0001552947",
    "https://www.sec.gov/cgi-bin/series?company=%09Series+Portfolios+Trust+&sc=companyseries&ticker=&CIK=&type=N-PX": "0001413032",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+SmartRetirement*+Blend+2035+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=GLOBAL+X+FUNDS": "0000029440",
    "https://www.sec.gov/cgi-bin/series?company=Impax+Funds+Series+Trust+I": "0000076721",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+CONTRAFUND&type=N-PX&sc=companyseries": "0000024238",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+VII": "0000318874",
    "https://www.sec.gov/cgi-bin/series?company=HUSSMAN+INVESTMENT+TRUST": "0001110502",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+CONCORD+STREET+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000819118",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Covington+Trust": "0000945908",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Advisor+Series+Equity+Growth+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0000722574",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000048027&type=N-PX": "0001618627",
    "https://www.sec.gov/cgi-bin/series?company=EQ+ADVISORS+TRUST": "0001027263",
    "https://www.sec.gov/cgi-bin/series?company=Mirae+Asset+Discovery+Funds&sc=companyseries&ticker=&CIK=&type=N-PX": "0001489215",
    "https://www.sec.gov/cgi-bin/series?company=EuroPac+International+Dividend+Income&type=N-PX&sc=companyseries": "0001318342",
    "https://www.sec.gov/cgi-bin/series?company=EuroPac+Gold+Fund&type=N-PX&sc=companyseries": "0001318342",
    "https://www.sec.gov/cgi-bin/series?company=Direxion+Nanotechnology+ETF": "0001424958",
    "https://www.sec.gov/cgi-bin/series?company=MORGAN+FLEMING+MUTUAL": "0001037897",
    "https://www.sec.gov/cgi-bin/series?company=IQ+Hedge+Event-Driven+Tracker+ETF": "0001415995",
    "https://www.sec.gov/cgi-bin/series?company=PRINCIPAL+VARIABLE+CONTRACTS+FUNDS%2C+INC": "0000012601",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+HIP+Sustainable+Muni+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=KraneShares+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001452937",
    "https://www.sec.gov/cgi-bin/series?company=Grandeur+Peak+Global+Micro+Cap+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0000915802",
    "https://www.sec.gov/cgi-bin/series?company=PFS+FUNDS&type=N-PX&sc=companyseries": "0001103243",
    "https://www.sec.gov/cgi-bin/series?company=Tidal+ETF+Trust+II": "0001924868",
    "https://www.sec.gov/cgi-bin/series?company=UBS+SERIES+FUNDS": "0001060517",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+MUNICIPAL+TRUST+II+": "0000878662",
    "https://www.sec.gov/cgi-bin/series?company=Tidal+ETF+Trust": "0001742912",
    "https://www.sec.gov/cgi-bin/series?company=Undiscovered+Managers+Behavioral+Value+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0001047712",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0000814679",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Series+Growth+Company+Fund": "0000707823",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000076091&type=N-PX": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=TIDAL+ETF+TRUST+II&sc=companyseries&ticker=&CIK=&type=N-PX": "0001924868",
    "https://www.sec.gov/cgi-bin/series?company=PUTNAM+INVESTMENT+FUNDS": "0001005942",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Tactical+Bond+Fund": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=%09Tidal+ETF+Trust+II": "0001924868",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+Exchange-Traded+Fund+Trust+II": "0001378872",
    "https://www.sec.gov/cgi-bin/series?company=Segall+Bryant+and+Hamill+Trust+": "0001318342",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000050657&type=N-PX": "0000063068",
    "https://www.sec.gov/cgi-bin/series?company=CARILLON+SERIES+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000897111",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+MSCI+Energy+Index+ETF&type=N-PX&sc=companyseries": "0000945908",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Rising+Dividend+Growth+Fund": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=GOLDMAN+SACHS+TRUST": "0000814679",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+EXCHANGE-TRADED+SELF": "0001657201",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000038775&type=N-PX": "0001330967",
    "https://www.sec.gov/cgi-bin/series?company=%09FIDELITY+HEREFORD+STREET+TRUST": "0000917286",
    "https://www.sec.gov/cgi-bin/series?company=Tidal+ETF+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001742912",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Strategic+Factor+Allocation+Fund": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+SAI+Sustainable+Core+Plus+Bond+Fund": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=Eubel+Brady": "0001606378",
    "https://www.sec.gov/cgi-bin/series?sc=companyseries&type=N-PX&company=HOMESTEAD+FUNDS+INC": "0000865733",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+VIP+Trust": "0000811976",
    "https://www.sec.gov/cgi-bin/series?company=Investment+Managers+Series+Trust+II&sc=companyseries&ticker=&CIK=&type=N-PX": "0001587982",
    "https://www.sec.gov/cgi-bin/series?sc=companyseries&type=N-PX&company=NORTHERN+LIGHTS+FUND+TRUST+II": "0001518042",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Short+Muni+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Series+Trust+XII&sc=companyseries&ticker=&CIK=&type=N-PX": "0001330967",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000076104&type=N-PX": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=AIM+COUNSELOR+SERIES+TRUST+%28INVESCO+COUNSELOR+SERIES+TRUST%29": "0001112996",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Managed+Income+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=%09Unified+Series+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001199046",
    "https://www.sec.gov/cgi-bin/series?company=Penn+Capital+Short+Duration+High+Income+Fund": "0001618627",
    "https://www.sec.gov/cgi-bin/series?company=The+National+Tax-Free+Intermediate+Bond+Fund&type=N-PX&sc=companyseries": "0000926243",
    "https://www.sec.gov/cgi-bin/series?company=First+Trust+Exchange-Traded+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0001383496",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+SmartRetirement+2055+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=HOMESTEAD+FUNDS%2C+INC": "0000865733",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+INCOME+FUND": "0000880195",
    "https://www.sec.gov/cgi-bin/series?company=AIM+GROWTH+SERIES+%28INVESCO+GROWTH+SERIES%29&type=N-PX&sc=companyseries": "0001112996",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000039735&type=N-PX": "0001537140",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Sustainable+High+Yield+ETF": "0000945908",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Pharmaceutical+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=UNIFIED+SERIES+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0001199046",
    "https://www.sec.gov/cgi-bin/series?company=Tidal+ETF+Trust+II&sc=companyseries&ticker=&CIK=&type=N-PX": "0001924868",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+HASTINGS+STREET+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000035348",
    "https://www.sec.gov/cgi-bin/series?company=PRINCIPAL+VARIABLE+CONTRACTS+FUNDS%2C+INC.&type=N-PX&sc=companyseries": "0000012601",
    "https://www.sec.gov/cgi-bin/series?company=Dimensional+ETF+Trust": "0001816125",
    "https://www.sec.gov/cgi-bin/series?company=Kraneshares+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001452937",
    "https://www.sec.gov/cgi-bin/series?company=Goldman%20Sachs%20Trust&CIK=&start=501&count=500": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=AIM+INTERNATIONAL+MUTUAL+FUNDS+%28INVESCO+INTERNATIONAL+MUTUAL+FUNDS%29&sc=companyseries&ticker=&CIK=&type=N-PX": "0000880859",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Corporate+Bond+Fund": "0001485894",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Fallen+Angel+High+Yield+Bond+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Tidal+ETF+Trust+II&type=N-PX&sc=companyseries": "0001924868",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+COVINGTON+TRUST": "0000945908",
    "https://www.sec.gov/cgi-bin/series?company=BRANDES+INVESTMENT+TRUST&type=N-PX&sc=companyseries": "0000926678",
    "https://www.sec.gov/cgi-bin/series?company=Tidal+ETF+Trust+II+-+REX+2X+AMC+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001924868",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Financial+Square+Money+Market+Fund": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=EA+SERIES+TRUST": "0001592900",
    "https://www.sec.gov/cgi-bin/series?company=Valkyrie+ETF+Trust+II&sc=companyseries&ticker=&CIK=&type=N-PX": "0001877493",
    "https://www.sec.gov/cgi-bin/series?company=Global+X+Funds+&sc=companyseries&ticker=&CIK=&type=N-PX": "0000029440",
    "https://www.sec.gov/cgi-bin/series?company=SHORT-TERM+INVESTMENTS+TRUST": "0000912035",
    "https://www.sec.gov/cgi-bin/series?company=AIM+COUNSELOR+SERIES+TRUST+%28INVESCO+COUNSELOR+SERIES+TRUST%29&type=N-PX&sc=companyseries": "0001112996",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+XII+-+MFS&type=N-PX&sc=companyseries": "0001330967",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+SAI+Sustainable+Emerging+Markets+Equity+Fund": "0000744822",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Short+High+Yield+Muni+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Two+Roads+Shared+Trust&type=N-PX&sc=companyseries": "0001552947",
    "https://www.sec.gov/cgi-bin/series?company=MFS+High+Yield+Pooled+Portfolio": "0000225604",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Access+Balanced+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=Frontier+MFG+Select+Infrastructure+Fund": "0001014913",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+SmartRetirement+Blend+2015+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000046813&type=N-PX": "0001414039",
    "https://www.sec.gov/cgi-bin/series?company=2nd+Vote+Funds&sc=companyseries&ticker=&CIK=&type=N-PX": "0001811168",
    "https://www.sec.gov/cgi-bin/series?company=Investment+Managers+Series+Trust+II": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=S000050364&CIK=&type=N-PX": "0001552947",
    "https://www.sec.gov/cgi-bin/series?company=GOLDMAN%20SACHS%20TRUST&CIK=&start=1&count=500": "0000814679",
    "https://www.sec.gov/cgi-bin/series?company=Investment+Managers+Series+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001318342",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Trust+II&sc=companyseries&ticker=&CIK=&type=N-PX": "0001580353",
    "https://www.sec.gov/cgi-bin/series?company=Touchstone+Strategic+Trust": "0001919700",
    "https://www.sec.gov/cgi-bin/series?company=Syntax+Stratified+LargeCap+ETF": "0001580843",
    "https://www.sec.gov/cgi-bin/series?company=DFA+INVESTMENT+DIMENSIONS+GROUP+INC&sc=companyseries&ticker=&CIK=&type=N-PX": "0000355437",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+CLO+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=JOHN+HANCOCK+VARIABLE+INSURANCE+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000756913",
    "https://www.sec.gov/cgi-bin/series?company=Capitol+Series+Trust": "0001587551",
    "https://www.sec.gov/cgi-bin/series?company=PRINCIPAL%20FUNDS%2C%20INC&CIK=&start=501&count=500": "0000898745",
    "https://www.sec.gov/cgi-bin/series?company=Unified+Series+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001199046",
    "https://www.sec.gov/cgi-bin/series?company=CONESTOGA+FUNDS": "0001175813",
    "https://www.sec.gov/cgi-bin/series?company=PRAXIS+MUTUAL+FUNDS&sc=companyseries&ticker=&CIK=&type=N-PX": "0000912900",
    "https://www.sec.gov/cgi-bin/series?company=DBX+ETF+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0001503123",
    "https://www.sec.gov/cgi-bin/series?company=WILLIAMSBURG+INVESTMENT+TRUST+": "0000842512",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+VIII": "0000819673",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+J.P.+Morgan+EM+Local+Currency+Bond+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Pioneer+Series+Trust+VII&sc=companyseries&ticker=&CIK=&type=N-PX": "0001140157",
    "https://www.sec.gov/cgi-bin/series?company=GOLDMAN+SACHS+ETF+TRUST+II&sc=companyseries&ticker=&CIK=&type=N-PX": "0001882879",
    "https://www.sec.gov/cgi-bin/series?company=Variable+Insurance+Products+Fund+V": "0000823535",
    "https://www.sec.gov/cgi-bin/series?company=Multi-Manager+U.S.+Small+Cap+Equity+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0001557156",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+NEW+YORK+MUNICIPAL+TRUST+II+": "0000878663",
    "https://www.sec.gov/cgi-bin/series?company=+INVESCO+EXCHANGE-TRADED+FUND+TRUST&type=N-PX&sc=companyseries": "0001595386",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+SmartRetirement+2060+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=Advisors%27+Inner+Circle+Fund+III&sc=companyseries&ticker=&CIK=&type=N-PX": "0001593547",
    "https://www.sec.gov/cgi-bin/series?company=Listed+Funds+Trust": "0001547950",
    "https://www.sec.gov/cgi-bin/series?company=Investment+Managers+Series+Trust": "0001318342",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+SmartRetirement+Blend+2060+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=GOLDMAN+SACHS+Financial+Square+Treasury+Instruments+Fund": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000030088&type=N-PX": "0001464413",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+REVERE+STREET+TRUST": "0001022695",
    "https://www.sec.gov/cgi-bin/series?company=Tema+ETF+Trust": "0001944285",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000072997&type=N-PX": "0001710607",
    "https://www.sec.gov/cgi-bin/series?company=GOLDMAN%20SACHS%20TRUST&CIK=&start=1001&count=500": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=ISHARES+TRUST&type=N-PX&sc=companyseries": "0000814679",
    "https://www.sec.gov/cgi-bin/series?company=Roundhill+Big+Defense+ETF": "0001683471",
    "https://www.sec.gov/cgi-bin/series?company=RMB+Investors+Trust": "0000030126",
    "https://www.sec.gov/cgi-bin/series?company=Simplify+Exchange+Traded+Funds": "0001810747",
    "https://www.sec.gov/cgi-bin/series?company=AIM+COUNSELOR+SERIES+TRUST+%28INVESCO+COUNSELOR+SERIES+TRUST%29+-+Invesco+Floating+Rate+ESG+Fund": "0001112996",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Dynamic+High+Income+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+Exchange-Traded+Fund+Trust+II&type=N-PX&sc=companyseries": "0001378872",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000077485&type=N-PX": "0001710607",
    "https://www.sec.gov/cgi-bin/series?company=Tactical+Investment+Series+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001719812",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000077486&type=N-PX": "0001710607",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000076100&type=N-PX": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=Hartford+Schroders+Tax-Aware+Bond+Fund": "0000049905",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000018679&type=N-PX": "0001060517",
    "https://www.sec.gov/cgi-bin/series?company=FAIRHOLME+FUNDS+INC": "0001096344",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+MERRIMACK+STREET+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0001562565",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Active+China+ETF&type=N-PX&sc=companyseries": "0001485894",
    "https://www.sec.gov/cgi-bin/series?company=Transamerica+Mid+Cap+Value": "0000787623",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000072998&type=N-PX": "0001710607",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+ETF+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001479026",
    "https://www.sec.gov/cgi-bin/series?company=%09FIDELITY+NEW+YORK+MUNICIPAL+TRUST+II": "0000878663",
    "https://www.sec.gov/cgi-bin/series?company=Abraham+Fortress+Fund&type=N-PX&sc=companyseries": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=Innovator+ETFs+Trust": "0001415726",
    "https://www.sec.gov/cgi-bin/series?company=TOUCHSTONE+FUNDS+GROUP+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000914243",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000038773&type=N-PX": "0001330967",
    "https://www.sec.gov/cgi-bin/series?company=RBB+Fund+Trust": "0001618627",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+ZERO+Total+Market+Index+Fund": "0000819118",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Preferred+Securities+ex+Financials+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Morningstar+International+Moat+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Orinda+Income+Opportunities+Fund&type=N-PX&sc=companyseries": "0001027596",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+INCOME+FUND%09&sc=companyseries&ticker=&CIK=&type=N-PX": "0000880195",
    "https://www.sec.gov/cgi-bin/series?company=COMMERCE+FUNDS&sc=companyseries&ticker=&CIK=&type=N-PX": "0000926243",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Russia+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=WISCONSIN+CAPITAL+FUNDS+INC&sc=companyseries&ticker=&CIK=&type=N-PX": "0001395397",
    "https://www.sec.gov/cgi-bin/series?sc=companyseries&type=N-PX&company=Fidelity+Greenwood+Street+Trust": "0001898391",
    "https://www.sec.gov/cgi-bin/series?company=EA+Series+Trust": "0001592900",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+ACTIVELY+MANAGED+EXCHANGE-TRADED+FUND+TRUST": "0001595386",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+Exchange-Traded+Self-Indexed+Fund+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001657201",
    "https://www.sec.gov/cgi-bin/series?company=Idaho+Tax-Exempt+Fund": "0000811860",
    "https://www.sec.gov/cgi-bin/series?company=Emerge+ETF+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001914404",
    "https://www.sec.gov/cgi-bin/series?company=FIRSTHAND+FUNDS&sc=companyseries&ticker=&CIK=&type=N-PX": "0000917124",
    "https://www.sec.gov/cgi-bin/series?company=%09GOLDMAN+SACHS+ETF+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0001479026",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+EXCHANGE-TRADED+FUND+TRUST+II&sc=companyseries&tickeakkkr=&CIK=&type=N-PX": "0001378872",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+INVESTMENT+TRUST": "0000354046",
    "https://www.sec.gov/cgi-bin/series?company=Simplify+Exchange+Traded+Funds+": "0001810747",
    "https://www.sec.gov/cgi-bin/series?company=%09VARIABLE+INSURANCE+PRODUCTS+FUND+IV&sc=companyseries&ticker=&CIK=&type=N-PX": "0000720318",
    "https://www.sec.gov/cgi-bin/series?company=DFA+INVESTMENT+DIMENSIONS+GROUP+INC&type=N-PX&sc=companyseries": "0000355437",
    "https://www.sec.gov/cgi-bin/series?company=DOUBLELINE+COMMERCIAL+REAL+ESTATE+ETF": "0001886172",
    "https://www.sec.gov/cgi-bin/series?company=DIAMOND+HILL+FUNDS": "0001032423",
    "https://www.sec.gov/cgi-bin/series?company=ETF+Series+Solutions&sc=companyseries&ticker=&CIK=&type=N-PX": "0001540305",
    "https://www.sec.gov/cgi-bin/series?company=State+Farm+Interim+Fund": "0001516523",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+Exchange-Traded+Fund+Trust+II&sc=companyseries&ticker=&CIK=&type=N-PX": "0001378872",
    "https://www.sec.gov/cgi-bin/series?company=AMERICAN+PENSION+INVESTORS+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000764859",
    "https://www.sec.gov/cgi-bin/series?company=DOUBLELINE+MORTGAGE+ETF": "0001886172",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000066146&type=N-PX": "0001492374",
    "https://www.sec.gov/cgi-bin/series?company=Strategic+Advisers+Fidelity+U.S.+Total+Stock+Fund": "0001364924",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+California+Municipal+Money+Market+Fund": "0000878662",
    "https://www.sec.gov/cgi-bin/series?company=Direxion+Daily+FinTech+Bull+2X+Shares&sc=companyseries&ticker=&CIK=&type=N-PX": "0001424958",
    "https://www.sec.gov/cgi-bin/series?company=ProShares+Metaverse+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001174610",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+SmartRetirement*+Blend+2045+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Green+Metals+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=GOLDMAN+SACHS+ETF+TRUST+II": "0001882879",
    "https://www.sec.gov/cgi-bin/series?company=Grandeur+Peak+US+Stalwarts+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0000915802",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Vietnam+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=INDEXIQ+ETF+TRUST": "0001364089",
    "https://www.sec.gov/cgi-bin/series?company=Quadratic+Deflation+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001547576",
    "https://www.sec.gov/cgi-bin/series?company=AIM+SECTOR+FUNDS": "0000725781",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Government+Cash+Reserves": "0000278001",
    "https://www.sec.gov/cgi-bin/series?company=GOLDMAN+SACHS+EMERGING+MARKETS+EQUITY+EX.+CHINA+FUND&type=N-PX&sc=companyseries": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=Vaneck+Long+Muni+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+EXCHANGE-TRADED+FUND+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0001595386",
    "https://www.sec.gov/cgi-bin/series?company=Master+Trust+-+ESG+Prime+Master+Fund": "0001403166",
    "https://www.sec.gov/cgi-bin/series?company=Tema+ETF+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001944285",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+SAI+Inflation-Focused+Fund": "0000028540",
    "https://www.sec.gov/cgi-bin/series?company=Penn+Capital+Opportunistic+High+Income+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0001618627",
    "https://www.sec.gov/cgi-bin/series?company=IMPAX+FUNDS+SERIES+TRUST+III": "0001598735",
    "https://www.sec.gov/cgi-bin/series?company=HOMESTEAD+FUNDS%2C+INC.": "0000865733",
    "https://www.sec.gov/cgi-bin/series?company=AIM+INVESTMENT+FUNDS+%28INVESCO+INVESTMENT+FUNDS%29+-+INVESCO+BALANCED-RISK+ALLOCATION+FUND&type=N-PX&sc=companyseries": "0000826644",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+ACTIVELY+MANAGED+EXCHANGE-TRADED+FUND+TRUST+-+Invesco+Variable+Rate+Investment+Grade+ETF": "0001418144",
    "https://www.sec.gov/cgi-bin/series?company=SIMPLIFY+EXCHANGE+TRADED+FUNDS&sc=companyseries&ticker=&CIK=&type=N-PX": "0001810747",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000066456&type=N-PX": "0001710607",
    "https://www.sec.gov/cgi-bin/series?company=FLEXSHARES+TRUST&type=N-PX&sc=companyseries": "0001491978",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+Exchange-Traded+Self-Indexed+Fund+Trust&type=N-PX&sc=companyseries": "0001657201",
    "https://www.sec.gov/cgi-bin/series?company=WILLIAMSBURG+INVESTMENT+TRUST": "0000842512",
    "https://www.sec.gov/cgi-bin/series?company=ASYMmetric+ETFs+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001833032",
    "https://www.sec.gov/cgi-bin/series?company=%09Unified+Series+Trust&sc=companyseries&ticker=&CIK=&typ": "0001199046",
    "https://www.sec.gov/cgi-bin/series?company=CONESTOGA+FUNDS&type=N-PX&sc=companyseries": "0001175813",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Indonesia+Index+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=AIM+SECTOR+FUNDS+%28INVESCO+SECTOR+FUNDS%29&sc=companyseries&ticker=&CIK=&type=N-PX": "0000725781",
    "https://www.sec.gov/cgi-bin/series?company=CLEARWATER+INVESTMENT+TRUST": "0000811161",
    "https://www.sec.gov/cgi-bin/series?company=Government+Portfolio": "0000709558",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Newbury+Street+Trust": "0000704207",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Series+Commodity+Strategy+Fund": "0000028540",
    "https://www.sec.gov/cgi-bin/series?company=DFA+INVESTMENT+DIMENSIONS+GROUP+INC.&type=N-PX&sc=companyseries": "0000355437",
    "https://www.sec.gov/cgi-bin/series?company=Harbor+Small+Cap+Explorer+ETF&type=N-PX&sc=companyseries": "0001860434",
    "https://www.sec.gov/cgi-bin/series?company=Nationwide+Diamond+Hill+Large+Cap+Concentrated+Fund": "0001048702",
    "https://www.sec.gov/cgi-bin/series?company=Northern+Lights+Fund+Trust+IV&sc=companyseries&ticker=&CIK=&type=N-PX": "0001537140",
    "https://www.sec.gov/cgi-bin/series?company=REX+2X+PTON+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001924868",
    "https://www.sec.gov/cgi-bin/series?company=Listed+Funds+Trust+-+Teucrium+AiLA+Long-Short+Agriculture+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001683471",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Large+Cap+Value+Insights+Fund": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000076657&type=N-PX": "0001919700",
    "https://www.sec.gov/cgi-bin/series?company=Tidal+ETF+Trust+II+-+REX+2X+NKLA+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001924868",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Emerging+Markets+Debt+Local+Currency+Central+Fund": "0000702533",
    "https://www.sec.gov/cgi-bin/series?company=Quadratic+Interest+Rate+Volatility+and+Inflation+Hedge+ETF": "0001540305",
    "https://www.sec.gov/cgi-bin/series?company=TRUST+FOR+ADVISED+PORTFOLIOS": "0001261788",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=S000028942&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+ESG+Emerging+Markets+Equity+Fund": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=Direxion+Shares+ETF+Trust": "0001424958",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Lifetime+2050+Fund": "0001330967",
    "https://www.sec.gov/cgi-bin/series?company=WILLIAMSBURG+INVESTMENT+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000842512",
    "https://www.sec.gov/cgi-bin/series?company=1919+Variable+Socially+Responsive+Balanced+Fund": "0001970751",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+India+Growth+Leaders+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=YieldMax+Innovation+Option+Income+Strategy+ETF": "0001924868",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000050658&type=N-PX": "0000783740",
    "https://www.sec.gov/cgi-bin/series?company=Fundvantage+Trust": "0001388485",
    "https://www.sec.gov/cgi-bin/series?company=TWO+ROADS+SHARED+TRUST&type=N-PX&sc=companyseries": "0001552947",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Sustainable+Core+Plus+Bond+ETF": "0001562565",
    "https://www.sec.gov/cgi-bin/series?company=NATIONWIDE+MUTUAL+FUNDS&sc=companyseries&ticker=&CIK=&type=N-PX": "0001048702",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Lifetime+2040+Fund": "0001330967",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Municipal+Income+Completion+Fund": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Energy+Income+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+SAI+Sustainable+Municipal+Income+Fund": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=ALLIANZ+VARIABLE+INSURANCE+PRODUCTS+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0001301708",
    "https://www.sec.gov/cgi-bin/series?company=HARTFORD+ULTRASHORT+BOND+HLS+FUND": "0001053425",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Environmental+Services+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Semiconductor+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+ETF+Trust+": "0001479026",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Israel+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=%09AIM+VARIABLE+INSURANCE+FUNDS+%28INVESCO+VARIABLE+INSURANCE+FUNDS%29": "0000896435",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Retail+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Sustainable+Emerging+Markets+Equity+Fund": "0000744822",
    "https://www.sec.gov/cgi-bin/series?company=Nationwide+American+Century+Small+Cap+Income+Fund": "0001048702",
    "https://www.sec.gov/cgi-bin/series?company=Wonderfund+Trust": "0001860923",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+U.S.+GARP+Equity+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Moody%27s+Analytics+IG+Corporate+Bond+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Direxion+Low+Priced+Stock+ETF": "0001424958",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+SAI+Sustainable+International+Equity+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0000744822",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000067047&type=N-PX": "0001552947",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Flex+Municipal+Income+Fund": "0000035373",
    "https://www.sec.gov/cgi-bin/series?company=HARTFORD+FUNDS+EXCHANGE-TRADED+TRUST": "0001501825",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Flex+Real+Estate+Fund": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=NEOS": "0001848758",
    "https://www.sec.gov/cgi-bin/series?company=Advocate+Rising+Rate+Hedge+ETF": "0001593547",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000076101&type=N-PX": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=Dimensional+International+Sustainability+Core+1+ETF": "0001816125",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Colchester+Street+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0000356173",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000006052&type=N-PX": "0000880709",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Morningstar+SMID+Moat+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Principal+Street+Short+Term+Municipal+Fund": "0001511699",
    "https://www.sec.gov/cgi-bin/series?company=IMPAX+LARGE+CAP+FUND&sc=companyseries&ticker=&CIK=&type=N-PX": "0000076721",
    "https://www.sec.gov/cgi-bin/series?company=%09INVESCO+EXCHANGE-TRADED+FUND+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0001595386",
    "https://www.sec.gov/cgi-bin/series?company=Dimensional+US+Core+Equity+Market+ETF": "0001816125",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+U.S.+Equity+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0001091439",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+Agriculture+Commodity+Strategy+No+K-1+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001595386",
    "https://www.sec.gov/cgi-bin/series?company=EuroPac+International+Bond+Fund&type=N-PX&sc=companyseries": "0001318342",
    "https://www.sec.gov/cgi-bin/series?company=AIM+VARIABLE+INSURANCE+FUNDS+%28INVESCO+VARIABLE+INSURANCE+FUNDS%29": "0000896435",
    "https://www.sec.gov/cgi-bin/series?company=Simplify+Exchange+Traded+Funds&sc=companyseries&ticker=&CIK=&type=N-PX": "0001810747",
    "https://www.sec.gov/cgi-bin/series?company=DFA+INVESTMENT+DIMENSIONS+GROUP+INC": "0000355437",
    "https://www.sec.gov/cgi-bin/series?company=Salient+Tactical+Plus+Fund": "0001535174",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000077487&type=N-PX": "0001710607",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+EXCHANGE-TRADED+FUND+TRUST+II": "0001378872",
    "https://www.sec.gov/cgi-bin/series?company=Eaton+Vance+NextShares+Trust": "0001573035",
    "https://www.sec.gov/cgi-bin/series?company=%09EQ+ADVISORS+TRUST+": "0001027263",
    "https://www.sec.gov/cgi-bin/series?company=Anfield+Universal+Fixed+Income+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001552947",
    "https://www.sec.gov/cgi-bin/series?company=CREDIT+SUISSE+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000941568",
    "https://www.sec.gov/cgi-bin/series?company=The+Value+Equity+Portfolio&sc=companyseries&ticker=&CIK=&type=N-PX": "0000934563",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Value+Advantage+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+Select+Risk%3A+Growth+Investor+Fund": "0000202032",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Aberdeen+Street+Trust": "0000880195",
    "https://www.sec.gov/cgi-bin/series?sc=companyseries&type=N-PX&company=GRANITESHARES+ETF+TRUST": "0001689873",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000076090&type=N-PX": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Colchester+Street+Trust": "0000356173",
    "https://www.sec.gov/cgi-bin/series?company=Cutler+Equity+Fund": "0000892568",
    "https://www.sec.gov/cgi-bin/series?company=Ultimus+Managers+Trust": "0001545440",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Advisor+Global+Equity+Income+Fund&type=N-PX&sc=companyseries": "0000729218",
    "https://www.sec.gov/cgi-bin/series?company=NORTHERN+LIGHTS+FUND+TRUST+II&sc=companyseries&ticker=&CIK=&type=N-PX": "0001518042",
    "https://www.sec.gov/cgi-bin/series?company=DIMENSIONAL+INVESTMENT+GROUP+INC": "0000355437",
    "https://www.sec.gov/cgi-bin/series?company=ProShares+On-Demand+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001174610",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Institutional+Trust": "0001303608",
    "https://www.sec.gov/cgi-bin/series?company=LoCorr+Dynamic+Opportunity+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0001506768",
    "https://www.sec.gov/cgi-bin/series?company=AIM+Tax-Exempt+Funds+%28Invesco+Tax-Exempt+Funds%29": "0000909466",
    "https://www.sec.gov/cgi-bin/series?company=GOLDMAN+SACHS+VARIABLE+INSURANCE+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000815425",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+Exchange-Traded+Self-Indexed+Fund+Trust+-+Invesco+BulletShares+2029+Corporate+Bond+ETF&type=N-PX&sc=companyseries": "0001657201",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+X&sc=companyseries&ticker=&CIK=&type=N-PX": "0000783740",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Massachusetts+Municipal+Money+Market+Fund": "0000357057",
    "https://www.sec.gov/cgi-bin/series?company=EuroPac+International+Value+Fund&type=N-PX&sc=companyseries": "0001318342",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+U.S.+Applied+Data+Science+Value+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=LINCOLN+VARIABLE+INSURANCE+PRODUCTS+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000914036",
    "https://www.sec.gov/cgi-bin/series?company=Dimensional+Global+Real+Estate+ETF": "0001816125",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+SECURITIES+FUND": "0000795422",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Series+International+Growth+Fund": "0000744822",
    "https://www.sec.gov/cgi-bin/series?sc=companyseries&type=N-PX&company=STONE+RIDGE+TRUST": "0001559992",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+V.I.+Core+Bond+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0000896435",
    "https://www.sec.gov/cgi-bin/series?company=THE+COMMERCE+FUNDS": "0000926243",
    "https://www.sec.gov/cgi-bin/series?company=ETF+Series+Solutions+&sc=companyseries&ticker=&CIK=&type=N-PX": "0001540305",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Commodity+Strategy+Central+Fund": "0000803013",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+SmartRetirement+2050+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000018680&type=N-PX": "0001060517",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Strategic+Volatility+Premium+Fund": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=State+Farm+Balanced+Fund": "0001516523",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+SmartRetirement+2025+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Growth+Strategies+K6+Fund": "0000707823",
    "https://www.sec.gov/cgi-bin/series?company=O%27Shares+Europe+Quality+Dividend+ETF": "0001479599",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000069564&type=N-PX": "0001815493",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+SAI+Sustainable+Conservative+Income+Municipal+Bond+Fund": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000076102&type=N-PX": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000076103&type=N-PX": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000058122&type=N-PX": "0000883622",
    "https://www.sec.gov/cgi-bin/series?company=Syntax+Stratified+MidCap+ETF&type=N-PX&sc=companyseries": "0001580843",
    "https://www.sec.gov/cgi-bin/series?company=Dimensional+US+Large+Cap+Value+ETF": "0001816125",
    "https://www.sec.gov/cgi-bin/series?company=Direxion+mRNA+ETF": "0001424958",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000058290&type=N-PX": "0000908406",
    "https://www.sec.gov/cgi-bin/series?company=KraneShares+Trust": "0001452937",
    "https://www.sec.gov/cgi-bin/series?company=Mutual+Fund+and+Variable+Insurance+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0000837274",
    "https://www.sec.gov/cgi-bin/series?company=Pacific+Select+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=JPMORGAN+TRUST+II&sc=companyseries&ticker=&CIK=&type=N-PX": "0001382990",
    "https://www.sec.gov/cgi-bin/series?company=AIM+TREASURER%27S+SERIES+TRUST+%28INVESCO+TREASURER%27S+SERIES+TRUST%29": "0000828806",
    "https://www.sec.gov/cgi-bin/series?company=Principal+Exchange-Traded+Funds": "0001572661",
    "https://www.sec.gov/cgi-bin/series?company=Tidal+ETF+Trust+II+-+REX+2X+TLRY+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001924868",
    "https://www.sec.gov/cgi-bin/series?company=Amplify+ETF+Trust&type=N-PX&sc=companyseries": "0001633061",
    "https://www.sec.gov/cgi-bin/series?company=State+Farm+Growth+Fund": "0001516523",
    "https://www.sec.gov/cgi-bin/series?company=HARBOR+ETF+TRUST&type=N-PX&sc=companyseries": "0001860434",
    "https://www.sec.gov/cgi-bin/series?company=Northern+Lights+Fund+Trust+IV+&sc=companyseries&ticker=&CIK=&type=N-PX": "0001537140",
    "https://www.sec.gov/cgi-bin/series?company=PRUDENTIAL+INVESTMENT+PORTFOLIOS+9&type=N-PX&sc=companyseries": "0001070287",
    "https://www.sec.gov/cgi-bin/series?company=HARBOR+FUNDS": "0000793769",
    "https://www.sec.gov/cgi-bin/series?company=Castle+Focus+Fund": "0001103243",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+EXCHANGE-TRADED+SELF-INDEXED+FUND+TRUST": "0001657201",
    "https://www.sec.gov/cgi-bin/series?company=AdvisorShares+Trust+": "0001408970",
    "https://www.sec.gov/cgi-bin/series?company=Dimensional+Emerging+Core+Equity+Market+ETF": "0001816125",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+U.S.+Sustainable+Leaders+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Aggressive+Growth+Allocation+Fund": "0000783740",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000063485&type=N-PX": "0001750821",
    "https://www.sec.gov/cgi-bin/series?company=REX+2X+GME+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001771146",
    "https://www.sec.gov/cgi-bin/series?sc=companyseries&type=N-PX&company=Ultimus+Managers+Trust": "0001545440",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Sustainable+Core+Plus+Bond+Fund&type=N-PX&sc=companyseries": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=THE+COMMERCE+FUNDS&sc=companyseries&ticker=&CIK=&type=N-PX": "0000926243",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+Short+Term+Bond+Fund&type=N-PX&sc=companyseries": "0000842790",
    "https://www.sec.gov/cgi-bin/series?company=WCM+China+Quality+Growth+Fund&type=N-PX&sc=companyseries": "0001318342",
    "https://www.sec.gov/cgi-bin/series?company=AIM+EQUITY+FUNDS": "0000105377",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000018681&type=N-PX": "0001060517",
    "https://www.sec.gov/cgi-bin/series?company=PRINCIPAL+VARIABLE+CONTRACTS+FUNDS+INC": "0000012601",
    "https://www.sec.gov/cgi-bin/series?company=Timothy+Plan+&sc=companyseries&ticker=&CIK=&type=N-PX": "0000916490",
    "https://www.sec.gov/cgi-bin/series?company=INVESTMENT+MANAGERS+SERIES+TRUST+II": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000067046&type=N-PX": "0001552947",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+SmartRetirement+Income+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?CIK=&sc=companyseries&type=N-PX&ticker=S000018300&Find=Search": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+U.S.+Research+Enhanced+Equity+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Lifetime+2030+Fund": "0001330967",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Video+Gaming+and+eSports+ETF": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=Aperture+International+Equity+Fund": "0001593547",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000076097&type=N-PX": "0001587982",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Tactical+Bond+ETF&type=N-PX&sc=companyseries": "0001562565",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Technology+Opportunities+Fund": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=S000072783&CIK=&type=N-PX": "0001552947",
    "https://www.sec.gov/cgi-bin/series?company=State+Farm+Municipal+Bond+Fund": "0001516523",
    "https://www.sec.gov/cgi-bin/series?company=Credit+Suisse+Commodity+Strategy+Funds&sc=companyseries&ticker=&CIK=&type=N-PX": "0001291446",
    "https://www.sec.gov/cgi-bin/series?company=PRAXIS+MUTUAL+FUNDS": "0000912900",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+California+Municipal+Money+Market+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0000878662",
    "https://www.sec.gov/cgi-bin/series?company=YieldMax+COIN+Option+Income+Strategy+ETF": "0001924868",
    "https://www.sec.gov/cgi-bin/series?company=Direxion+Daily+Oil+Services+Bull+2X+Shares": "0001424958",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Mid+Cap+Equity+Fund": "0001485894",
    "https://www.sec.gov/cgi-bin/series?company=AIM+GROWTH+SERIES+%28INVESCO+GROWTH+SERIES%29": "0001112996",
    "https://www.sec.gov/cgi-bin/series?company=FundVantage+Trust": "0001388485",
    "https://www.sec.gov/cgi-bin/series?company=MassMutual+Select+T.+Rowe+Price+U.S.+Treasury+Long-Term+Index+Fund": "0000916053",
    "https://www.sec.gov/cgi-bin/series?company=Invesco+Global+Targeted+Returns+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0000826644",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Target+Date+2060+Portfolio": "0001557156",
    "https://www.sec.gov/cgi-bin/series?company=HARBOR+ETF+TRUST": "0001860434",
    "https://www.sec.gov/cgi-bin/series?company=TRANSAMERICA+FUNDS": "0001018953",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+XII&sc=companyseries&ticker=&CIK=&type=N-PX": "0001330967",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Merrimack+Street+Trust": "0001562565",
    "https://www.sec.gov/cgi-bin/series?company=Hartford+Schroders+Emerging+Markets+Multi-Sector+Bond+Fund&type=N-PX&sc=companyseries": "0000049905",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=S000067385&CIK=&type=N-PX": "0001355064",
    "https://www.sec.gov/cgi-bin/series?company=SCHWARTZ+INVESTMENT+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0000891160",
    "https://www.sec.gov/cgi-bin/series?company=Arrow+Investments+Trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001527428",
    "https://www.sec.gov/cgi-bin/series?company=VanEck+Green+Infrastructure+ETF&sc=companyseries&ticker=&CIK=&type=N-PX": "0001137360",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Federal+Money+Market+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=Sector+Rotation+Fund": "0001364089",
    "https://www.sec.gov/cgi-bin/series?company=Polen+DDJ+Opportunistic+High+Yield+Fund": "0001558107",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000054778&type=N-PX": "0000351601",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Real+Estate+Equity+Central+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0001303459",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Rutland+Square+Trust+II+": "0001364924",
    "https://www.sec.gov/cgi-bin/series?company=NEUBERGER+BERMAN+EQUITY+FUNDS&sc=companyseries&ticker=&CIK=&type=N-PX": "0001317474",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Tactical+Tilt+Overlay+Fund": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=Dimensional+Global+Sustainability+Fixed+Income+ETF": "0001816125",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Prime+Money+Market+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Focused+Value+Fund": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=Northern%20Lights%20Fund%20Trust&CIK=&start=501&count=500": "0001314414",
    "https://www.sec.gov/cgi-bin/series?company=Neuberger+Berman+Advisers+Management+Trust": "0000736913",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Central+Investment+Portfolios+LLC": "0001401097",
    "https://www.sec.gov/cgi-bin/series?company=Zacks+All-Cap+Core+Fund&type=N-PX&sc=companyseries": "0001318342",
    "https://www.sec.gov/cgi-bin/series?company=PD+Small-Cap+Growth+Index+Portfolio": "0000813900",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Rutland+Square+Trust+II": "0001364924",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+CHARLES+STREET+TRUST": "0000354046",
    "https://www.sec.gov/cgi-bin/series?company=The+Chesapeake+Growth+Fund": "0000893759",
    "https://www.sec.gov/cgi-bin/series?company=Dimensional+US+Sustainability+Core+1+ETF": "0001816125",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+REVERE+STREET+TRUST&sc=companyseries&ticker=&CIK=&type=N-PX": "0001022695",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+School+Street+Trust": "0000215829",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Concord+Street+Trust+&sc=companyseries&ticker=&CIK=&type=N-PX": "0000819118",
    "https://www.sec.gov/cgi-bin/series?company=%09Infusive+US+Trust+": "0001748197",
    "https://www.sec.gov/cgi-bin/series?company=Dimensional+U.S.+Small+Cap+ETF": "0001816125",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Advisor+Multi-Asset+Income+Fund": "0000215829",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Sustainable+Intermediate+Municipal+Income+Fund": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+XIV": "0001392417",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Total+Market+Index+Fund": "0000819118",
    "https://www.sec.gov/cgi-bin/series?company=MFS+VARIABLE+INSURANCE+TRUST+II&type=N-PX&sc=companyseries": "0000719269",
    "https://www.sec.gov/cgi-bin/series?company=Lord+Abbett+Diversification+Shares": "0001139819",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Income+Builder+Fund": "0001485894",
    "https://www.sec.gov/cgi-bin/series?company=Transamerica+Total+Return": "0000787623",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Conservative+Allocation+Fund": "0000783740",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Global+Governments+Portfolio": "0000719269",
    "https://www.sec.gov/cgi-bin/series?company=MFS+VARIABLE+INSURANCE+TRUST": "0001091439",
    "https://www.sec.gov/cgi-bin/series?company=THE+HARTFORD+GROWTH+OPPORTUNITIES+FUND": "0000049905",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+IX&sc=companyseries&ticker=&CIK=&type=N-PX": "0000063075",
    "https://www.sec.gov/cgi-bin/series?company=proshares+trust&sc=companyseries&ticker=&CIK=&type=N-PX": "0001174610",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+New+Millennium+Fund": "0000707823",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Emerging+Markets+Equity+Central+Fund": "0001303459",
    "https://www.sec.gov/cgi-bin/series?company=Hartford+Emerging+Markets+Equity+Fund": "0000049905",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Emerging+Markets+Debt+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=MFS+International+Growth+Portfolio": "0000356476",
    "https://www.sec.gov/cgi-bin/series?CIK=&sc=companyseries&type=N-PX&ticker=S000009318&Find=Search": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=ProShares+Ultra+Technology": "0001174610",
    "https://www.sec.gov/cgi-bin/series?company=Pioneer+Global+Multisector+Income+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0001140157",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+DEVONSHIRE+TRUST": "0000035341",
    "https://www.sec.gov/cgi-bin/series?company=AQR+Diversified+Arbitrage+Fund": "0001444822",
    "https://www.sec.gov/cgi-bin/series?company=GOLDMAN+SACHS+VARIABLE+INSURANCE+TRUST": "0000815425",
    "https://www.sec.gov/cgi-bin/series?company=Ivy+Variable+Insurance+Portfolios": "0000810016",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Municipal+Income+Fund": "0000795422",
    "https://www.sec.gov/cgi-bin/series?company=HARTFORD+TOTAL+RETURN+BOND+HLS+FUND": "0001053425",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Convertible+Securities+Fund": "0000708191",
    "https://www.sec.gov/cgi-bin/series?company=&sc=companyseries&ticker=&CIK=S000014299&type=N-PX": "0001174610",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+VI": "0000863032",
    "https://www.sec.gov/cgi-bin/series?company=GOLDMAN+SACHS+VARIABLE+INSURANCE+TRUST&type=N-PX&sc=companyseries": "0000815425",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+I": "0000933691",
    "https://www.sec.gov/cgi-bin/series?company=The+Hartford+Emerging+Markets+Local+Debt+Fund": "0001006415",
    "https://www.sec.gov/cgi-bin/series?company=Hartford+Multi-Asset+Income+Fund": "0001006415",
    "https://www.sec.gov/cgi-bin/series?company=RUSSELL+INVESTMENT": "0000351601",
    "https://www.sec.gov/cgi-bin/series?company=MFS+International+Equity+Fund": "0000063068",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Total+Return+Series": "0000063075",
    "https://www.sec.gov/cgi-bin/series?company=Hartford+International+Equity+Fund": "0001006415",
    "https://www.sec.gov/cgi-bin/series?company=Strategic+Advisers+Short+Duration+Fund": "0001364924",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Global+Growth+Fund": "0000819673",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Total+Return+Bond+Fund": "0000063075",
    "https://www.sec.gov/cgi-bin/series?company=Biotechnology+Portfolio": "0001062417",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+ADVISOR+SERIES+VIII+": "0000729218",
    "https://www.sec.gov/cgi-bin/series?company=MASSMUTUAL+SELECT+FUNDS&sc=companyseries&ticker=&CIK=&type=N-PX": "0000916053",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Emerging+Markets+Equity+Fund": "0000063068",
    "https://www.sec.gov/cgi-bin/series?company=PRUDENTIAL+SERIES+FUND&type=N-PX&sc=companyseries": "0000711175",
    "https://www.sec.gov/cgi-bin/series?company=Variable+Insurance+Products+Fund+V+": "0000823535",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+HASTINGS+STREET+TRUST": "0000035348",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+Europe+Dynamic+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=HARTFORD+CAPITAL+APPRECIATION+HLS+FUND": "0000741565",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Growth+Allocation+Fund": "0000783740",
    "https://www.sec.gov/cgi-bin/series?company=THE+HARTFORD+CAPITAL+APPRECIATION+FUND": "0001006415",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+International+Equity+Fund": "0001485894",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Global+High+Yield+Fund": "0000225604",
    "https://www.sec.gov/cgi-bin/series?company=THE+HARTFORD+BALANCED+INCOME+FUND": "0001006415",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Growth+Series": "0000814679",
    "https://www.sec.gov/cgi-bin/series?company=HARTFORD+SMALL+COMPANY+HLS+FUND": "0001053425",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Equity+Dividend+Income+Fund": "0000708191",
    "https://www.sec.gov/cgi-bin/series?company=AIM+INVESTMENT+SECURITIES+FUNDS+%28INVESCO+INVESTMENT+SECURITIES+FUNDS%29": "0000826644",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+II&type=N-PX&sc=companyseries": "0001413032",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Connecticut+Municipal+Income+Fund&type=N-PX&sc=companyseries": "0000225323",
    "https://www.sec.gov/cgi-bin/series?company=THE+HARTFORD+FLOATING+RATE+FUND": "0001006415",
    "https://www.sec.gov/cgi-bin/series?company=MFS+New+Discovery+Value+Fund": "0000356349",
    "https://www.sec.gov/cgi-bin/series?company=%09FIDELITY+PURITAN+TRUST": "0000081205",
    "https://www.sec.gov/cgi-bin/series?company=GMO+SGM+Major+Markets+Fund": "0001521894",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+ADVISOR+SERIES+VIII&sc=companyseries&ticker=&CIK=&type=N-PX": "0000729218",
    "https://www.sec.gov/cgi-bin/series?company=AQR+International+Momentum+Style+Fund": "0001444822",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+V.I.+High+Yield+Fund&sc=companyseries&ticker=&CIK=&type=N-PX": "0000896435",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Charles+Street+Trust": "0000354046",
    "https://www.sec.gov/cgi-bin/series?company=HARTFORD+SMALL+CAP+VALUE+FUND": "0001006415",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+New+York+Municipal+Income+Fund": "0000718581",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Municipal+High+Income+Fund": "0000225604",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Growth+Strategies+Fund": "0000707823",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Leveraged+Company+Stock+Fund": "0000722574",
    "https://www.sec.gov/cgi-bin/series?company=MFS+International+Growth+Fund": "0000356476",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Growth+Company+Fund": "0000707823",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Michigan+Municipal+Income+Fund&type=N-PX&sc=companyseries": "0000035373",
    "https://www.sec.gov/cgi-bin/series?company=ProShares+Ultra+SmallCap600": "0001174610",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Global+Real+Estate+Fund": "0000356349",
    "https://www.sec.gov/cgi-bin/series?company=HARTFORD+CORE+EQUITY+FUND": "0001006415",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+ABERDEEN+STREET+TRUST": "0000880195",
    "https://www.sec.gov/cgi-bin/series?company=MFS+Blended+Research+Core+Equity+Fund": "0000911637",
    "https://www.sec.gov/cgi-bin/series?company=JPMorgan+National+Municipal+Income+Fund": "0001217286",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+COMMONWEALTH+TRUST": "0000205323",
    "https://www.sec.gov/cgi-bin/series?company=FIDELITY+ADVISOR+SERIES+I": "0000722574",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Intermediate+Municipal+Income+Fund&type=N-PX&sc=companyseries": "0000035315",
    "https://www.sec.gov/cgi-bin/series?company=Goldman+Sachs+Enhanced+Dividend+Global+Equity+Portfolio": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Trend+Fund": "0000035402",
    "https://www.sec.gov/cgi-bin/series?company=%09FIDELITY+HANOVER+STREET+TRUST": "0000702533",
    "https://www.sec.gov/cgi-bin/series?CIK=&sc=companyseries&type=N-PX&ticker=S000020773&Find=Search": "0000822977",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Arizona+Municipal+Income+Fund": "0000035330",
    "https://www.sec.gov/cgi-bin/series?company=+HARTFORD+HEALTHCARE+HLS+FUND": "0001053425",
    "https://www.sec.gov/cgi-bin/series?company=Fidelity+Limited+Term+Municipal+Income+Fund": "0000035373",
    "https://www.sec.gov/cgi-bin/series?company=HARTFORD+BALANCED+HLS+FUND": "0001053425",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+XV": "0000764719",
    "https://www.sec.gov/cgi-bin/series?company=MFS+SERIES+TRUST+I+": "0000933691",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009252&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000007493&scd=series": "0000320351",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000029320&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009306&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000018567&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003037&scd=series": "0001209466",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000028298&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000010511&scd=series": "0000812015",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000072999&scd=series": "0001710607",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000035024&scd=series": "0000826644",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000026039&scd=series": "0000813900",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000056849&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000055539&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000015278&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000069577&scd=series": "0000813900",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009354&scd=series": "0001046292",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000026601&scd=series": "0000914243",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000028302&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000026598&scd=series": "0000914243",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000026013&scd=series": "0000813900",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000036460&scd=series": "0000813900",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000056167&scd=series": "0000914243",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000008063&scd=series": "0001112996",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000001887&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000066458&scd=series": "0001710607",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000001906&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000063664&scd=series": "0001587551",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000007848&scd=series": "0000044402",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000034530&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000049922&scd=series": "0000915802",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000000220&scd=series": "0000205007",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000067086&scd=series": "0001199046",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000076660&scd=series": "0001919700",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000000218&scd=series": "0000019034",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000074252&scd=series": "0001199046",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000034881&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000073398&scd=series": "0001501825",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000065568&scd=series": "0000793769",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000052136&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000030663&scd=series": "0000896435",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000027889&scd=series": "0000896435",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000036643&scd=series": "0001418144",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000019231&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000025062&scd=series": "0000793769",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000063935&scd=series": "0001587551",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003049&scd=series": "0001209466",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000027881&scd=series": "0000896435",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000065065&scd=series": "0001103243",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000001898&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000072516&scd=series": "0001137360",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000048866&scd=series": "0000813900",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003056&scd=series": "0001209466",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000038520&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000066535&scd=series": "0001479026",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000000229&scd=series": "0000880859",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000030967&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000077919&scd=series": "0001683471",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000066454&scd=series": "0001710607",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000047581&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000075973&scd=series": "0000225322",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000027890&scd=series": "0000896435",
    "https://www.sec.gov/cgi-bin/series?company=INVESCO+EXCHANGE-TRADED+FUND+TRUST+II&sc=companyseries&ticker=&CIK=&type=N-PX": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000032769&scd=series": "0001209466",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000005840&scd=series": "0000914243",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000073970&scd=series": "0001437249",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009340&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000070151&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000000684&scd=series": "0000793769",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000037359&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000034748&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000000155&scd=series": "0000725781",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000044081&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000058242&scd=series": "0001645194",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003059&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000037368&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000026045&scd=series": "0000813900",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000058241&scd=series": "0001645194",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000055915&scd=series": "0001199046",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003073&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000002290&scd=series": "0001062806",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003990&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000027370&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000070646&scd=series": "0001408970",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000060111&scd=series": "0001552947",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000055846&scd=series": "0000076721",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000026032&scd=series": "0000813900",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000004531&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000033429&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000000213&scd=series": "0000019034",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000075157&scd=series": "0001816125",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000075148&scd=series": "0001816125",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000026017&scd=series": "0000813900",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000075152&scd=series": "0001816125",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000075154&scd=series": "0001816125",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000027873&scd=series": "0000896435",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000066459&scd=series": "0001710607",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000008842&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003967&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000071403&scd=series": "0000813900",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000034882&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000000193&scd=series": "0000896435",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000039927&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000065628&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000029069&scd=series": "0001437249",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000061192&scd=series": "0001452937",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000034585&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000002794&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003966&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000019230&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000049413&scd=series": "0001006415",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000030111&scd=series": "0000826644",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003072&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000070432&scd=series": "0001261788",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000024098&scd=series": "0001041673",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000043249&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003036&scd=series": "0001209466",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003968&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000000476&scd=series": "0000202032",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000013111&scd=series": "0001209466",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000052134&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003075&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000007210&scd=series": "0000898745",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003951&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000069264&scd=series": "0001261788",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000037370&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014037&scd=series": "0001168164",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000041596&scd=series": "0001547576",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000073559&scd=series": "0001816125",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000060147&scd=series": "0001314414",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000070901&scd=series": "0001816125",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000000252&scd=series": "0000842790",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003989&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000000172&scd=series": "0000909466",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003953&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003079&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000019245&scd=series": "0001378872",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000034880&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000071543&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000073562&scd=series": "0001816125",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000077595&scd=series": "0001848758",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000027871&scd=series": "0000896435",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003074&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000071541&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000027869&scd=series": "0000909466",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000046566&scd=series": "0001511699",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003986&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003981&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000001901&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000056358&scd=series": "0000035315",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000039948&scd=series": "0001437249",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000002460&scd=series": "0000716716",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000069657&scd=series": "0001261788",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000054486&scd=series": "0001388485",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000049412&scd=series": "0001006415",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000055845&scd=series": "0000076721",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003068&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000037366&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003972&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003076&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000075151&scd=series": "0001816125",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000026043&scd=series": "0000813900",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000020098&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000012103&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000012056&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003064&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000022994&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003976&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003964&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000002948&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003955&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003984&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000004469&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000001894&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000004527&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000002660&scd=series": "0000719269",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003960&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003982&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003109&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000019761&scd=series": "0000889519",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003080&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000020314&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000030505&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009255&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000018829&scd=series": "0001401097",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000020739&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000004871&scd=series": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000012117&scd=series": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003099&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003065&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003965&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000024912&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009241&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000024919&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000007467&scd=series": "0000320351",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014276&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000012879&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000028919&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000024911&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000031635&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000006823&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000004876&scd=series": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000022292&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000006824&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000004880&scd=series": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009342&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000036933&scd=series": "0001401097",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000022843&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000032548&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000013795&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000022299&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000024909&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014302&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000015787&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000018728&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000024918&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000030249&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000024910&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009304&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000004873&scd=series": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014252&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009305&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000029317&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000018721&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000004884&scd=series": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000037860&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000030517&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000002571&scd=series": "0000752177",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009303&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000012119&scd=series": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000026012&scd=series": "0000813900",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014306&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000012118&scd=series": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014262&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014254&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000004881&scd=series": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014282&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009307&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000018733&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000018724&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000006831&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000018740&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000035008&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000004866&scd=series": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009308&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000028921&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000018718&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000020741&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003078&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000008846&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003956&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003082&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000006821&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009300&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003959&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014307&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003961&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000022290&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009309&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000008849&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000018723&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000024913&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003085&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014310&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000022304&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003111&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000009301&scd=series": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000033079&scd=series": "0001137761",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014253&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000006828&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003100&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003104&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000024920&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014263&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000002615&scd=series": "0001217286",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003067&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014300&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014257&scd=series": "0001174610",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000024296&scd=series": "0000898745",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003980&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003107&scd=series": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000028523&action=getcompany&scd=filings": "0001469192",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000069094&action=getcompany&scd=filings": "0001364924",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000007134&action=getcompany&scd=filings": "0000880799",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000061413&type=485&dateb=&count=40&scd=filings&search_text=": "0000822977",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000062807&action=getcompany&scd=filings": "0000917286",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000070442&action=getcompany&scd=filings": "0001342947",
    "https://www.sec.gov/cgi-bin/browse-edgar?company=AIM+VARIABLE+INSURANCE+FUNDS+%28INVESCO+VARIABLE+INSURANCE+FUNDS%29&match=starts-with&filenum=&State=&Country=&SIC=&myowner=exclude&action=getcompany": "0000896435",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014039&type=485&dateb=&count=40&scd=filings&search_text=": "0001168164",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000063843&action=getcompany&scd=filings": "0001559109",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000062265&action=getcompany&scd=filings": "0001364924",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000006749&action=getcompany&scd=filings": "0000357204",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000040740&type=485&dateb=&count=40&scd=filings&search_text=": "0001168164",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000022173&action=getcompany&scd=filings": "0000105377",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000045671&owner=include&scd=filings&count=40": "0000898745",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000000277&action=getcompany&scd=filings": "0000105377",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000006753&action=getcompany&scd=filings": "0000357204",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000072296&action=getcompany&scd=filings": "0001479026",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000071430&action=getcompany&scd=filings": "0001479026",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000052163&type=485&dateb=&count=40&scd=filings&search_text=": "0001516212",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000077356&action=getcompany&scd=filings": "0001657201",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000003084&action=getcompany&scd=filings": "0001039803",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000065033&type=485&dateb=&count=40&scd=filings&search_text=": "0001540305",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000000185&action=getcompany&scd=filings": "0000896435",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000028539&owner=include&scd=filings&count=40": "0001469192",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000073019": "0001657201",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000012960&owner=include&scd=filings&count=40": "0000355437",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000029151&action=getcompany&scd=filings": "0001364924",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000006744&action=getcompany&scd=filings": "0000357204",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000003156&owner=include&scd=filings&count=40": "0001032423",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000072294&action=getcompany&scd=filings": "0001479026",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000030092&action=getcompany&scd=filings": "0001364924",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014040&type=485&dateb=&count=40&scd=filings&search_text=": "0001168164",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000001085&owner=include&scd=filings&count=40": "0000796229",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000021764&type=485&dateb=&count=40&scd=filings&search_text=": "0001168164",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014038&type=485&dateb=&count=40&scd=filings&search_text=": "0001168164",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000062891&owner=include&scd=filings&count=40": "0000877670",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000001065&owner=include&scd=filings&count=40": "0000909504",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&filenum=001-36170&owner=exclude&count=40": "0001580185",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000029153&action=getcompany&scd=filings": "0001364924",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000014246&owner=include&scd=filings&count=40": "0000898745",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000008762&owner=include&scd=filings&count=40": "0000870355",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000007136&action=getcompany&scd=filings": "0000880799",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000063758&scd=serie": "0000357204",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000008771&owner=include&scd=filings&count=40": "0000870355",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000007053&action=getcompany&scd=filings": "0000917286",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000015909&action=getcompany&scd=filings": "0001364923",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000092494&owner=include&scd=filings&count=40": "0001479026",
    "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000092493&owner=include&scd=filings&count=40": "0001479026",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000008406&action=getcompany&scd=filings": "0001303459",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000004874&action=getcompany&scd=filings": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000002210&action=getcompany&scd=filings": "0000711175",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000008404&action=getcompany&scd=filings": "0001303459",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000004883&action=getcompany&scd=filings": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000004882&action=getcompany&scd=filings": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000015910&action=getcompany&scd=filings": "0001364923",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000015911&action=getcompany&scd=filings": "0001364923",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000020954&action=getcompany&scd=filings": "0001303459",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000004869&action=getcompany&scd=filings": "0000880195",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000027862&action=getcompany&scd=filings": "0000202032",
    "https://www.sec.gov/cgi-bin/browse-edgar?CIK=S000009353&action=getcompany&scd=filings": "0001046292",
}

In [7]:
DATA_CIK = re.compile(r"/data/([0-9]+)/")
CIK_EQUALS = re.compile(r"[\&\?]CIK=([0-9]+)")

def find_cik(url):
    m = DATA_CIK.search(url)
    if m:
        return m.group(1)
    m = CIK_EQUALS.search(url)
    if m:
        return m.group(1)
    if url in webpage_to_cik:
        return webpage_to_cik[url]
    return pd.NA

In [8]:
some_relationships = relationships[relationships["Registration.ValidationReference"].str.startswith("https://www.sec.gov").fillna(False)]
cik = some_relationships["Registration.ValidationReference"].apply(find_cik)

/tmp/ipykernel_81781/1149851067.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  some_relationships = relationships[relationships["Registration.ValidationReference"].str.startswith("https://www.sec.gov").fillna(False)]


In [50]:
len(relationships), len(some_relationships)

(457501, 9187)

In [21]:
lei_cik_mapping = some_relationships["Relationship.EndNode.NodeID"].to_frame("LEI").assign(CIK=cik)
lei_cik_mapping

,LEI,CIK
0,5493001Z012YSB2A0K51,722574
1,C7J4FOV6ELAVE39B7M82,0000722574
8,549300GC5MDF1KXYMP06,1458460
39,CYSD4L2598RQ9BPPCB72,831001
40,6SHGI4ZSSLCXXQSBB395,831001
...,...,...
457473,HCHV7422L5HDJZCRFL38,97745
457474,HCHV7422L5HDJZCRFL38,97745
457475,JKUQM1EGY6VH6BM0ZI63,1121403
457494,5493001Z012YSB2A0K51,354046


In [55]:
len(lei_cik_mapping.drop_duplicates(["CIK"]))

2006

In [10]:
np.count_nonzero(cik.notna()), len(cik)

(np.int64(9155), 9187)

In [23]:
sec = pd.read_csv("sec-cik-data.csv", dtype={"cik": str}).rename(columns={"cik": "CIK"})
sec

,filename,CIK,name,address,city,state,zip,ticker,exchange,subsidiaries
0,03/000095017023011133.htm,0001750149,"INHIBIKASE THERAPEUTICS, INC.",3350 Riverwood Parkway SE,Atlanta,GA,30339,IKT,The Nasdaq Stock Market LLC,NaN
1,03/000143774923007024.htm,0001375195,THE CORETEC GROUP INC.,600 S. Wagner Rd,Ann Arbor,MI,48103,NaN,NaN,NaN
2,03/000162828023006564.htm,0001726445,"SEER, INC.",3800 Bridge Parkway,Redwood City,California,94065,SEER,NASDAQ Global Select Market,NaN
3,03/000095017023007989.htm,0001678660,PRELUDE THERAPEUTICS INC,200 Powder Mill Road,Wilmington,Delaware,19803,PRLD,The Nasdaq Stock Market LLC,NaN
4,03/000119312523058346.htm,0001705843,"Calyxt, Inc.",2800 Mount Ridge Road,Roseville,MN,55113-1127,CLXT,The NASDAQ Stock Market LLC,NaN
...,...,...,...,...,...,...,...,...,...,...
7976,02/000170940123000005.htm,0001709401,"RUBIUS THERAPEUTICS, INC.","c/o Verdolino & Lowey, P.C.","Foxborough,",Massachusetts,02035,RUBY,The Nasdaq Global Select Market,NaN
7977,02/000102312823000053.htm,0001023128,LITHIA MOTORS INC,"150 N. Bartlett Street,","Medford,",Oregon,97501,LAD,The New York Stock Exchange,NaN
7978,02/000089101423000014.htm,0000891014,MINERALS TECHNOLOGIES INC.,622 Third Avenue,New York,New York,10017-6707,MTX,New York Stock Exchange LLC,exhibit21_1.htm
7979,02/000103046923000010.htm,0001030469,OFG Bancorp,254 Muñoz Rivera Avenue,San Juan,NaN,00918,OFG,New York Stock Exchange,NaN


In [27]:
matched_lei = lei_cik_mapping.merge(registry, on="LEI")
matched_lei

,LEI,CIK,Entity.LegalName,Entity.LegalName.xmllang,Entity.OtherEntityNames.OtherEntityName.1,Entity.OtherEntityNames.OtherEntityName.1.xmllang,Entity.OtherEntityNames.OtherEntityName.1.type,Entity.OtherEntityNames.OtherEntityName.2,Entity.OtherEntityNames.OtherEntityName.2.xmllang,Entity.OtherEntityNames.OtherEntityName.2.type,...,Registration.OtherValidationAuthorities.OtherValidationAuthority.3.ValidationAuthorityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.3.OtherValidationAuthorityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.3.ValidationAuthorityEntityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.4.ValidationAuthorityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.4.OtherValidationAuthorityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.4.ValidationAuthorityEntityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.5.ValidationAuthorityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.5.OtherValidationAuthorityID,Registration.OtherValidationAuthorities.OtherValidationAuthority.5.ValidationAuthorityEntityID,ConformityFlag
0,5493001Z012YSB2A0K51,722574,FIDELITY MANAGEMENT & RESEARCH COMPANY LLC,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONFORMING
1,C7J4FOV6ELAVE39B7M82,0000722574,FIDELITY ADVISOR SERIES I,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONFORMING
2,549300GC5MDF1KXYMP06,1458460,"AQR CAPITAL MANAGEMENT, LLC",en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONFORMING
3,CYSD4L2598RQ9BPPCB72,831001,CITIBANK OVERSEAS INVESTMENT CORPORATION,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONFORMING
4,6SHGI4ZSSLCXXQSBB395,831001,CITIGROUP INC.,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONFORMING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9182,HCHV7422L5HDJZCRFL38,97745,THERMO FISHER SCIENTIFIC INC.,en,Thermo Electron Corporation,en,PREVIOUS_LEGAL_NAME,Thermo Electron Engineering Corporation,en,PREVIOUS_LEGAL_NAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONFORMING
9183,HCHV7422L5HDJZCRFL38,97745,THERMO FISHER SCIENTIFIC INC.,en,Thermo Electron Corporation,en,PREVIOUS_LEGAL_NAME,Thermo Electron Engineering Corporation,en,PREVIOUS_LEGAL_NAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONFORMING
9184,JKUQM1EGY6VH6BM0ZI63,1121403,"BRACEBRIDGE CAPITAL, LLC",en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONFORMING
9185,5493001Z012YSB2A0K51,354046,FIDELITY MANAGEMENT & RESEARCH COMPANY LLC,en,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONFORMING


In [28]:
matched = matched_lei.merge(sec, on="CIK", how="outer", indicator=True)
matched

,LEI,CIK,Entity.LegalName,Entity.LegalName.xmllang,Entity.OtherEntityNames.OtherEntityName.1,Entity.OtherEntityNames.OtherEntityName.1.xmllang,Entity.OtherEntityNames.OtherEntityName.1.type,Entity.OtherEntityNames.OtherEntityName.2,Entity.OtherEntityNames.OtherEntityName.2.xmllang,Entity.OtherEntityNames.OtherEntityName.2.type,...,filename,name,address,city,state,zip,ticker,exchange,subsidiaries,_merge
0,NaN,0000001750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,07/000110465923082069.htm,AAR CORP,One AAR Place,Wood Dale,Illinois,60191,AIR,New York Stock Exchange,NaN,right_only
1,HQD377W2YR662HK5JX27,0000001800,ABBOTT LABORATORIES,en,NaN,NaN,NaN,NaN,NaN,NaN,...,02/000162828023004026.htm,Abbott Laboratories,100 Abbott Park Road,Abbott Park,Illinois,60064-6400,ABT,New York Stock Exchange,NaN,both
2,HQD377W2YR662HK5JX27,0000001800,ABBOTT LABORATORIES,en,NaN,NaN,NaN,NaN,NaN,NaN,...,02/000162828023004026.htm,Abbott Laboratories,100 Abbott Park Road,Abbott Park,Illinois,60064-6400,ABT,New York Stock Exchange,NaN,both
3,NaN,0000001961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,03/000126493123000006.htm,WORLDS INC.,11 Royal Road,Brookline,MA,02445,NaN,NaN,NaN,right_only
4,NaN,0000002098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,03/000156459023003422.htm,ACME UNITED CORP,1 Waterview Drive,Shelton,Connecticut,06484,ACU,NYSE American,NaN,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24092,549300Q4QMDLLFX8BQ98,<NA>,SHP ETF Trust,en,NaN,NaN,NaN,NaN,NaN,NaN,...,02/000035089423000017.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
24093,549300Q4QMDLLFX8BQ98,<NA>,SHP ETF Trust,en,NaN,NaN,NaN,NaN,NaN,NaN,...,02/000168316823000978.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
24094,549300Q4QMDLLFX8BQ98,<NA>,SHP ETF Trust,en,NaN,NaN,NaN,NaN,NaN,NaN,...,02/000082002723000016.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
24095,549300Q4QMDLLFX8BQ98,<NA>,SHP ETF Trust,en,NaN,NaN,NaN,NaN,NaN,NaN,...,02/000121390023014746.htm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [31]:
len(matched.query("_merge == 'left_only'"))

9083

In [32]:
len(matched.query("_merge == 'right_only'"))

7703

In [33]:
len(matched.query("_merge == 'both'"))

7311

In [35]:
7311 + 7703 + 9083 == 24097

True

In [36]:
both_matched = matched.query("_merge == 'both'")

In [39]:
both_matched[[
    "Entity.LegalName",
    "Entity.LegalAddress.FirstAddressLine", "Entity.LegalAddress.City", "Entity.LegalAddress.Region", "Entity.LegalAddress.PostalCode",
#    "Entity.HeadquartersAddress.FirstAddressLine", "Entity.HeadquartersAddress.City", "Entity.HeadquartersAddress.Region", "Entity.HeadquartersAddress.PostalCode",
    "name", "address", "city", "state", "zip", "ticker", "exchange"
]]

,Entity.LegalName,Entity.LegalAddress.FirstAddressLine,Entity.LegalAddress.City,Entity.LegalAddress.Region,Entity.LegalAddress.PostalCode,name,address,city,state,zip,ticker,exchange
1,ABBOTT LABORATORIES,C/O C T CORPORATION SYSTEM,CHICAGO,US-IL,60604,Abbott Laboratories,100 Abbott Park Road,Abbott Park,Illinois,60064-6400,ABT,New York Stock Exchange
2,ABBOTT LABORATORIES,C/O C T CORPORATION SYSTEM,CHICAGO,US-IL,60604,Abbott Laboratories,100 Abbott Park Road,Abbott Park,Illinois,60064-6400,ABT,New York Stock Exchange
347,GENERAL DYNAMICS CORPORATION,CORPORATION TRUST CENTER 1209 ORANGE ST,WILMINGTON,US-DE,19801,GENERAL DYNAMICS CORPORATION,11011 Sunset Hills Road,"Reston,",Virginia,20190,GD,New York Stock Exchange
348,GENERAL ELECTRIC COMPANY,1 NEUMANN WAY,EVENDALE,US-OH,45215,GENERAL ELECTRIC COMPANY,5 Necco Street,Boston,MA,02210,GE,New York Stock Exchange
349,GENERAL ELECTRIC COMPANY,1 NEUMANN WAY,EVENDALE,US-OH,45215,GENERAL ELECTRIC COMPANY,5 Necco Street,Boston,MA,02210,GE,New York Stock Exchange
...,...,...,...,...,...,...,...,...,...,...,...,...
24092,SHP ETF Trust,CORPORATION TRUST CENTER 1209 ORANGE ST,Wilmington,US-DE,19801,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24093,SHP ETF Trust,CORPORATION TRUST CENTER 1209 ORANGE ST,Wilmington,US-DE,19801,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24094,SHP ETF Trust,CORPORATION TRUST CENTER 1209 ORANGE ST,Wilmington,US-DE,19801,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24095,SHP ETF Trust,CORPORATION TRUST CENTER 1209 ORANGE ST,Wilmington,US-DE,19801,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
print(f"{'LEI':20s} | {'GLEIF name':60s} | {'CIK':10s} | {'U.S. SEC name':40s}")
print(f"{'-' * 20}-+-{'-' * 60}-+-{'-' * 10}-+-{'-' * 40}")
seen = set()
for _, row in both_matched.iterrows():
    if row["LEI"] not in seen and isinstance(row["CIK"], str):
        print(f"{row['LEI']:20s} | {row['Entity.LegalName']:60s} | {row['CIK']:10s} | {row['name']:40s}")
    seen.add(row["LEI"])

print(len(seen))

LEI                  | GLEIF name                                                   | CIK        | U.S. SEC name                           
---------------------+--------------------------------------------------------------+------------+-----------------------------------------
HQD377W2YR662HK5JX27 | ABBOTT LABORATORIES                                          | 0000001800 | Abbott Laboratories                     
9C1X8XOOTYY2FNYTVH06 | GENERAL DYNAMICS CORPORATION                                 | 0000040533 | GENERAL DYNAMICS CORPORATION            
3C7474T6CDKPR9K6YT90 | GENERAL ELECTRIC COMPANY                                     | 0000040545 | GENERAL ELECTRIC COMPANY                
549300XMP3KDCKJXIU47 | MARSH & MCLENNAN COMPANIES, INC.                             | 0000062709 | Marsh & McLennan Companies, Inc.        
549300ZFEEJ2IP5VME73 | STATE STREET CORPORATION                                     | 0000093751 | STATE STREET CORPORATION                
WHENKOULSSK7WUM60H03